In [1]:
import pandas as pd
from pathlib import Path
from typing import Tuple
import functools
from pandas.core.groupby import DataFrameGroupBy
import itertools

In [2]:
CHECK_MAPPING = {
    "-1": "NOT SURE",
    "0": "NOT RELEVANT: BAD",
    "1": "RELEVANT",
    "2": "RELEVANT ENOUGH",
    "3": "NOT RELEVANT: USED THE FIELD, NOT CONTRIBUTING",
    "4": "NOT RELEVANT: FROM OTHER FIELD, NOT SEEING LINKS, LIKE ",
    "5": "NOT RELEVANT: IN SURROUNDING FIELD, NOT RELEVANT ENOUGH, LIKE VISUALIZATION TOOLS ETC.",
}

DATA_PATH = Path("./xlsx")
IEEE_PATH = DATA_PATH / "ieee-code_clone|detection-metadata.xlsx"
ACM_PATH = DATA_PATH / "acm-code_clone|detection-abstract.xlsx"

DUP_SIGNATURE_FIELDS = [
    ("DOI", "DOI"),
    ("ISBNs", "ISBN"),
    ("ISSN", "ISSN"),
    ("Abstract", "Abstract Note"),
    ("Document Title", "Title"),
]

In [3]:
def read_xlsx_and_group_by_check(path: Path) -> Tuple[pd.DataFrame, DataFrameGroupBy]:
    excel_data = pd.read_excel(path, dtype=str)
    excel_data.fillna("", inplace=True)
    groups = excel_data.groupby("check")

    for key in groups.indices:
        assert key in CHECK_MAPPING, f"index {key} not in check mapping"

    return excel_data, groups

In [4]:
def summarize_xlsx_group(grouped: DataFrameGroupBy) -> None:
    for check_flag, check_meaning in CHECK_MAPPING.items():
        try:
            group = grouped.get_group(check_flag)
        except KeyError:
            print(f"{check_meaning}: no data")
        else:
            print(f"{check_meaning}: {len(group)}")

In [5]:
def report_empty_in_field(data: pd.DataFrame, field: str) -> pd.DataFrame:
    return data[data[field] == ""]

In [6]:
def dedup_by_field(
    table1: pd.DataFrame, table2: pd.DataFrame, field1: str = "DOI", field2: str = None
) -> pd.DataFrame:
    if field2 is None:
        field2 = field1

    intersection = table1.merge(
        table2[[field2]],
        left_on=table1[field1].str.lower(),
        right_on=table2[field2].str.lower(),
        how="inner",
    )

    if field2 == field1:
        intersection = (
            intersection[
                ~(
                    (intersection[f"{field1}_x"] == "")
                    | (intersection[f"{field2}_y"] == "")
                )
            ]
            .drop(columns=[f"{field2}_y"])
            .rename(columns={f"{field1}_x": field1})
        )
    else:
        intersection = intersection[
            ~((intersection[field1] == "") | (intersection[field2] == ""))
        ].drop(columns=[field2])

    return intersection.drop(columns=["key_0"]).drop_duplicates()


def dedup_by_fields(table1, table2, fields):
    return functools.reduce(
        (lambda left, right: pd.concat([left, right])),
        map(lambda t: dedup_by_field(table1, table2, *t), fields),
    ).drop_duplicates()

In [7]:
ieee_data, grouped_ieee_data = read_xlsx_and_group_by_check(IEEE_PATH)
summarize_xlsx_group(grouped_ieee_data)

NOT SURE: 3
NOT RELEVANT: BAD: 26
RELEVANT: 208
RELEVANT ENOUGH: 18
NOT RELEVANT: USED THE FIELD, NOT CONTRIBUTING: 49
NOT RELEVANT: FROM OTHER FIELD, NOT SEEING LINKS, LIKE : 1
NOT RELEVANT: IN SURROUNDING FIELD, NOT RELEVANT ENOUGH, LIKE VISUALIZATION TOOLS ETC.: 2


In [8]:
acm_data, grouped_acm_data = read_xlsx_and_group_by_check(ACM_PATH)
summarize_xlsx_group(grouped_acm_data)

NOT SURE: no data
NOT RELEVANT: BAD: 6
RELEVANT: 58
RELEVANT ENOUGH: 9
NOT RELEVANT: USED THE FIELD, NOT CONTRIBUTING: 22
NOT RELEVANT: FROM OTHER FIELD, NOT SEEING LINKS, LIKE : 1
NOT RELEVANT: IN SURROUNDING FIELD, NOT RELEVANT ENOUGH, LIKE VISUALIZATION TOOLS ETC.: 1


In [9]:
# see if there are duplicated ACM entries in IEEE
# we filter those out before manually filtering
group_dedup = {
    group_id: dedup_by_fields(
        grouped_ieee_data.get_group(group_id)
        if group_id in grouped_ieee_data.groups
        else pd.DataFrame(index=ieee_data.index, columns=ieee_data.columns),
        grouped_acm_data.get_group(group_id)
        if group_id in grouped_acm_data.groups
        else pd.DataFrame(index=acm_data.index, columns=acm_data.columns),
        DUP_SIGNATURE_FIELDS,
    )
    for group_id in CHECK_MAPPING.keys()
}

In [10]:
different_group_dedup = dict(
    filter(
        (lambda t: len(t[1]) > 0),
        (
            (
                (group_id_1, group_id_2),
                dedup_by_fields(
                    grouped_ieee_data.get_group(group_id_1)
                    if group_id_1 in grouped_ieee_data.groups
                    else pd.DataFrame(index=ieee_data.index, columns=ieee_data.columns),
                    grouped_acm_data.get_group(group_id_2)
                    if group_id_2 in grouped_acm_data.groups
                    else pd.DataFrame(index=acm_data.index, columns=acm_data.columns),
                    DUP_SIGNATURE_FIELDS,
                ),
            )
            for group_id_1, group_id_2 in itertools.product(
                CHECK_MAPPING.keys(), CHECK_MAPPING.keys()
            )
            if group_id_1 != group_id_2
        ),
    )
)

In [11]:
invalid_dup_list = functools.reduce(
    lambda left, right: pd.concat([left, right]),
    (v for v in different_group_dedup.values()),
).drop_duplicates()
print(f"invalid dup list: {len(invalid_dup_list)}")

invalid dup list: 17


In [15]:
ieee_not_sure = (
    grouped_ieee_data.get_group("-1")
    if "-1" in grouped_ieee_data.groups
    else pd.DataFrame(index=ieee_data.index, columns=ieee_data.columns).dropna()
)
acm_not_sure = (
    grouped_acm_data.get_group("-1")
    if "-1" in grouped_acm_data.groups
    else pd.DataFrame(index=acm_data.index, columns=acm_data.columns).dropna()
)

print(f"ieee not sure: {len(ieee_not_sure)}")
print(f"acm not sure: {len(acm_not_sure)}")

ieee not sure: 3
acm not sure: 0
